In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from data import data_reader

Tensor = torch.Tensor
rand_m = np.random.random

In [2]:
MODEL_NAME = 'resnet34_cos'
if not os.path.exists('weights/'):
    !mkdir weights
if not os.path.exists('weights/{}'.format(MODEL_NAME)):
    !mkdir weights/{MODEL_NAME}

In [3]:
class model_resnet(nn.Module):
    def __init__(self, num_class):
        super(model_resnet, self).__init__()
        self.resnet = models.resnet34(pretrained=True)
        self.argp = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_class)
    
    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.argp(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [4]:
model = model_resnet(6).to('cuda')

In [5]:
init_lr = 0.001
n_cycle = 6
epoch_per_cycle = 10
cycle_add = 1.5
def cos_annealing_lr(initial_lr, cur_epoch, epoch_per_cycle):
    return initial_lr * (np.cos(np.pi * cur_epoch / epoch_per_cycle) + 1) / 2

In [6]:
optim = torch.optim.Adam(model.parameters(), lr=init_lr)
loss_func = torch.nn.CrossEntropyLoss()

In [7]:
from data import data_reader
class args:
    bs = 128
    path = 'train2000/'

In [8]:
dr = data_reader(args, 'train2000/train2000.csv')

In [9]:
train_loader, test_loader = dr.get_train_loader()

In [10]:
train_loss_rec = []
train_acc_rec = []
test_loss_rec = []
test_acc_rec = []
best_acc = 0
lr_list = []
for cycle in range(n_cycle):
    print('Snatshot# ', cycle)
    epoch_per_cycle += cycle_add
    for epoch in range(int(epoch_per_cycle)):
        epoch_loss = 0
        total, correct = 0, 0
        lr = cos_annealing_lr(init_lr, epoch, epoch_per_cycle)
        optim.state_dict()['param_groups'][0]['lr'] = lr
        lr_list.append(lr)
        print('\rCycle {} | Epoch # {} | lr = {}'.format(cycle, epoch, lr))
        for batch_idx, (X_train, y_train) in enumerate(train_loader):
            X_train, y_train = X_train.to('cuda'), y_train.to('cuda')
            optim.zero_grad()
            y_hat = model(X_train)
            loss = loss_func(y_hat, torch.max(y_train, 1)[1])
            loss.backward()
            _, pred = torch.max(y_hat.data, 1)
            optim.step()
            correct += (pred == torch.max(y_train, 1)[1]).sum().item()
            total += X_train.size(0)
            epoch_loss += loss.item() / len(train_loader)
            print('\rCycle {} | Epoch {} | Batch # {} Train Loss {:.5f} '.format(cycle, epoch, batch_idx, loss.item()))
        print('\rCycle {} | Epoch {} | Epoch Train Loss {:.5f}'.format(cycle, epoch, epoch_loss))
        epoch_acc = correct / total * 100
        print('\nCycle {} | Epoch {} | Epoch Train Acc {:.3f}%'.format(cycle, epoch, epoch_acc))
        train_loss_rec.append(epoch_loss)
        train_acc_rec.append(epoch_acc)
        with torch.no_grad():
            test_epoch_loss = 0
            test_total = 0
            test_correct = 0
            
            for batch_idx, (X_val, y_val) in enumerate(test_loader):
                X_val, y_val = X_val.to('cuda'), y_val.to('cuda')
            
                y_hat = model(X_val)
                loss = loss_func(y_hat, torch.max(y_val, 1)[1])
                _, pred = torch.max(y_hat.data, 1)
                print(pred[:5])
                print(pred.shape)
                test_total += y_val.size(0)
                test_correct += (pred == torch.max(y_val, 1)[1]).sum().item()
                test_epoch_loss += loss.item() / len(test_loader)
             
            test_epoch_acc = test_correct / test_total * 100
            if test_epoch_acc > best_acc:
                best_acc = test_epoch_acc
                torch.save(model.state_dict(), 'weights/' + MODEL_NAME + '/best_params_acc{}.pth'.format(best_acc)) 
            print('Cycle {} | Epoch {} | Epoch Val Loss {:.5f}'.format(cycle, epoch, test_epoch_loss))
            print('Cycle {} | Epoch {} | Epoch Val Acc {:.3f}%'.format(cycle, epoch, test_epoch_acc))
            
            test_loss_rec.append(test_epoch_loss)
            test_acc_rec.append(test_epoch_acc)
    cycle_add *= 1.5

Snatshot#  0
Cycle 0 | Epoch # 0 | lr = 0.001
Cycle 0 | Epoch 0 | Batch # 0 Train Loss 2.09168 
Cycle 0 | Epoch 0 | Batch # 1 Train Loss 0.51603 
Cycle 0 | Epoch 0 | Batch # 2 Train Loss 0.52453 
Cycle 0 | Epoch 0 | Batch # 3 Train Loss 0.58248 
Cycle 0 | Epoch 0 | Batch # 4 Train Loss 0.30847 
Cycle 0 | Epoch 0 | Batch # 5 Train Loss 1.36876 
Cycle 0 | Epoch 0 | Batch # 6 Train Loss 0.66849 
Cycle 0 | Epoch 0 | Batch # 7 Train Loss 0.63172 
Cycle 0 | Epoch 0 | Batch # 8 Train Loss 0.63823 
Cycle 0 | Epoch 0 | Batch # 9 Train Loss 0.51112 
Cycle 0 | Epoch 0 | Batch # 10 Train Loss 0.43191 
Cycle 0 | Epoch 0 | Batch # 11 Train Loss 0.33320 
Cycle 0 | Epoch 0 | Batch # 12 Train Loss 0.44817 
Cycle 0 | Epoch 0 | Epoch Train Loss 0.69652

Cycle 0 | Epoch 0 | Epoch Train Acc 76.375%
tensor([2, 5, 4, 4, 4], device='cuda:0')
torch.Size([128])
tensor([5, 5, 4, 1, 1], device='cuda:0')
torch.Size([128])
tensor([3, 5, 1, 4, 4], device='cuda:0')
torch.Size([128])
tensor([3, 5, 1, 4, 4], device='cu

Process Process-95:
Traceback (most recent call last):
  File "/data/tennant/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/data/tennant/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/data/tennant/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/data/tennant/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/data/tennant/data-competitions/dianshi_TG/data.py", line 40, in __getitem__
    img = self.transform(img)
  File "/data/tennant/anaconda3/envs/pytorch/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/data/ten

KeyboardInterrupt: 

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.subplot(2,1,1)
plt.plot(train_loss_rec, 'b')
plt.plot(test_loss_rec, 'r')

plt.title('loss')
plt.xlabel('Epoch')

plt.legend(['Train loss', 'Validation loss'], loc=4)

plt.subplot(2,1,2)
plt.plot(train_acc_rec, 'b')
plt.plot(test_acc_rec, 'r')

plt.title('Prediction Accuracy')
plt.xlabel('Epoch')

plt.legend(['Train prediction accuracy' ,'Validation prediction accuracy'], loc=4)
plt.show()

In [ ]:
plt.plot(lr_list)
plt.xlabel('epoch')
plt.ylabel('lr')
plt.show()